In [1]:
def get_input(filename, part=1):
     with open(filename, "r") as f:
          in1, in2 = f.read().split("\n\n")
          grid = {}
          for i, line in enumerate(in1.splitlines()):
               if part == 2:
                    line = (
                         line
                         .replace("#", "##")
                         .replace(".", "..")
                         .replace("O", "[]")
                         .replace("@", "@.")
                    )
               for j, s in enumerate(line):
                    grid[i, j] = s
                    if s == "@":
                         robot = (i, j)
     moves = [s for s in in2 if s != "\n"]
     return grid, moves, robot

In [2]:
def print_grid(grid):
    i_max = max(x[0] for x in grid)
    j_max = max(x[1] for x in grid)
    for i in range(i_max + 1):
        print("".join([grid[i, j] for j in range(j_max + 1)]))
        
def boxes(i, j, di, dj, grid):
    n_boxes = 0
    n = 1
    while True:
        ni, nj = i + n * di, j + n * dj
        if grid[ni, nj] == ".":
            return True, n_boxes
        if grid[ni, nj] == "#":
            return False, n_boxes
        if grid[ni, nj] == "O":
            n_boxes += 1
            n += 1

def boxes_lr(i, j, dj, grid):
    n_boxes = 0
    n = 1
    while True:
        nj = j + n * dj
        if grid[i, nj] == ".":
            return True, n_boxes
        if grid[i, nj] == "#":
            return False, n_boxes
        if grid[i, nj] == "[":
            n_boxes += 1
            n += 2

def boxes_up(i, j, grid):
    if grid[i - 1, j] == ".":
        return True, [(0, 0)]
    up1 = boxes_up(i - 1, j, grid)
    if grid[i - 1, j] == "]":
        up2 = boxes_up(i - 1, j - 1, grid)
    elif grid(i - 1, j) == "[":
        up2 = boxes_up(i - 1, j + 1, grid)
    return up1[0] and up2[0], [(0, 0)] + up1[1] + up2[1]


In [3]:
grid, moves, robot = get_input("15_input.txt")
dirs = {"<": (0, -1), "^": (-1, 0), ">": (0, 1), "v": (1, 0)}
i, j = robot
for move in moves:
    di, dj = dirs[move]
    can_move, n_boxes = boxes(i, j, di, dj, grid)
    if not can_move:
        continue
    grid[i + di, j + dj] = "@"
    grid[i, j] = "."
    if n_boxes > 0:
        grid[i + (n_boxes + 1) * di, j + (n_boxes + 1) * dj] = "O"
    i += di
    j += dj

gps = 0
for (i, j), s in grid.items():
    if s == "O":
        gps += i * 100 + j
print(gps)

1485257


In [4]:
grid, moves, robot = get_input("15_testinput2.txt", part=2)

In [5]:
print_grid(grid)

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
